In [9]:
import openeo
import xarray as xr

ImportError: C extension: pandas.compat not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

In [6]:
#connection = openeo.connect('https://openeocloud.vito.be/openeo/1.0.0').authenticate_oidc()
connection = openeo.connect("openeo.dataspace.copernicus.eu/openeo/1.2").authenticate_oidc()

Authenticated using refresh token.


In [3]:
latlonIDs = [[54.3167,10.05,7504.0], [54.4333,9.8167,7521.0], [53.95,10.6833,7539.0]]


In [4]:
def datacube_location_list(coords_list, IDs_list):
    locs_for_datacube =   { "type": "FeatureCollection",
        "features": []
    }
    for coord_index, coords in enumerate(coords_list):
        locs_for_datacube["features"].append({"type": "Feature", "properties": {'name':IDs_list[coord_index]}, "geometry": {"type": "Point", "coordinates": coords[:2]}})
    return locs_for_datacube

In [7]:
latlonIDs

[[54.3167, 10.05, 7504.0], [54.4333, 9.8167, 7521.0], [53.95, 10.6833, 7539.0]]

In [8]:
for location in latlonIDs:
    #locs_for_datacube = {"type": "Point", "properties": {'name':int(location[2])}, "coordinates": location[:2]}
    locs_for_datacube = datacube_location_list([location], [int(location[2])])
    datacube = connection.load_collection(
        "SENTINEL2_L2A",
        #spatial_extent={"west": 5, "south": 47, "east": 16, "north": 56},
        temporal_extent=["2019-01-01", "2019-01-31"],
        bands=["B04", "B08"]
        )
    locationed = datacube.filter_bbox([location[1] -0.01, location[0] - 0.01, location[1] + 0.01, location[0] + 0.01])
    #res = locationed.save_result(format = "NetCDF", options = {'filename_prefix':f'Station_satellites_{location[2]}'})
    job = locationed.create_job(title=f'Station_satellites_{int(location[2])}',
        description="Sentinel-2 L2A bands",
        out_format="NetCDF",
        options = {'filename_prefix':f'Station_satellites_{int(location[2])}'}
        )
    #job = res.execute_batch(title=f'Station_satellites_{location[2]}')
    job.start()

OpenEoApiError: [400] ConcurrentJobLimit: Job was not started because concurrent job limit (2) is reached. (ref: r-2412112ac3dc4876b522b779f45de42e)

In [5]:
for location in latlonIDs:
    #locs_for_datacube = {"type": "Point", "properties": {'name':int(location[2])}, "coordinates": location[:2]}
    locs_for_datacube = datacube_location_list([location], [int(location[2])])
    datacube = connection.load_collection(
        "SENTINEL2_L2A",
        #spatial_extent={"west": 5, "south": 47, "east": 16, "north": 56},
        temporal_extent=["2019-01-01", "2019-01-31"],
        bands=["B04", "B08"]
        )
    locationed = datacube.aggregate_spatial(geometries = locs_for_datacube, reducer = 'mean')
    #res = locationed.save_result(format = "NetCDF", options = {'filename_prefix':f'Station_satellites_{location[2]}'})
    job = locationed.create_job(title=f'Station_satellites_{int(location[2])}',
        description="Sentinel-2 L2A bands",
        out_format="NetCDF",
        options = {'filename_prefix':f'Station_satellites_{int(location[2])}'}
        )
    #job = res.execute_batch(title=f'Station_satellites_{location[2]}')
    job.start()#get_results().download_files("C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\downloads")

OpenEoApiError: [401] TokenExchangeFailure: Authentication failed. Please login on our marketplace web https://portal.terrascope.be first, and then try authenticate again. (ref: r-241211b6073445f9a115009ec236808e)